# Models

In this section we’ll take a closer look at creating and using a model. We’ll use the TFAutoModel class, which is handy when you want to instantiate any model from a checkpoint.

The TFAutoModel class and all of its relatives are actually simple wrappers over the wide variety of models available in the library. It’s a clever wrapper as it can automatically guess the appropriate model architecture for your checkpoint, and then instantiates a model with this architecture.

However, if you know the type of model you want to use, you can use the class that defines its architecture directly. Let’s take a look at how this works with a BERT model.

## Creating a Transformer

The first thing we’ll need to do to initialize a BERT model is load a configuration object: